In [1]:
from backend.dendron.out_go import process_file
from langchain.document_loaders import PDFPlumberLoader

# Define the path to the file, and other parameters
file_path = "/mnt/d/ai/cognimesh/output/katoAnticoagulationStrategiesExtracorporeal2021.pdf"
chunk_size = 1000
chunk_overlap = 50

# Now call the process_file function
process_file(path=file_path,
             loader_class=PDFPlumberLoader,
             chunk_size=chunk_size,
             chunk_overlap=chunk_overlap)



File with SHA1 79daea53f994b5d3a2eb7b4406e0527589cdc246 is already processed.


In [1]:
import unittest
from unittest.mock import patch, MagicMock
from backend.plugins.zotero_connect import ZoteroConnect

class TestZoteroConnectPlugin(unittest.TestCase):
    def setUp(self):
        self.plugin = ZoteroConnect()

    @patch('backend.plugins.zotero_connect.store')
    def test_setup_with_no_tables(self, mock_db_module):
        # Set up the mock for the database connection and cursor
        mock_connection = MagicMock()
        mock_cursor = MagicMock()
        
        # Simulate the database and cursor behavior
        mock_db_module.connect.return_value = mock_connection
        mock_connection.cursor.return_value = mock_cursor
        
        # Simulate that the tables do not exist and mock the result of querying table structure
        mock_cursor.fetchall.side_effect = [
            [],  # First call returns empty list indicating tables do not exist
            [
                ('doc_id',), 
                ('doi',), 
                ('metadata',)
            ],  # Second call simulates the result of "SELECT column_name FROM information_schema.columns WHERE table_name='zotero_articles'"
            [
                ('doc_id',), 
                ('isbn',), 
                ('metadata',)
            ],  # Third call simulates the result of "SELECT column_name FROM information_schema.columns WHERE table_name='zotero_books'"
        ]
        
        # Call the setup method of the plugin
        self.plugin.setup(config={})
        
        # Check that the cursor executed the table creation SQL
        create_table_calls = [
            unittest.mock.call("""
                CREATE TABLE IF NOT EXISTS zotero_articles (
                    id SERIAL PRIMARY KEY,
                    doc_id UUID NOT NULL,
                    doi VARCHAR(255),
                    metadata JSONB,
                    FOREIGN KEY (doc_id) REFERENCES documents(doc_id)
                );
            """),
            unittest.mock.call("""
                CREATE TABLE IF NOT EXISTS zotero_books (
                    id SERIAL PRIMARY KEY,
                    doc_id UUID NOT NULL,
                    isbn VARCHAR(255),
                    metadata JSONB,
                    FOREIGN KEY (doc_id) REFERENCES documents(doc_id)
                );
            """)
        ]
        mock_cursor.execute.assert_has_calls(create_table_calls, any_order=True)

        # Clean up
        mock_cursor.close.assert_called()
        mock_connection.commit.assert_called()

    # More tests...

# Instantiate the test suite with all tests from the above test class
test_suite = unittest.TestLoader().loadTestsFromTestCase(TestZoteroConnectPlugin)

# Run the test suite
test_runner = unittest.TextTestRunner(verbosity=2)
test_runner.run(test_suite)

test_setup_with_no_tables (__main__.TestZoteroConnectPlugin) ... 2023-11-10 08:18:06,880 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 08:18:06,885 - zotero_connect - ERROR - Table structure for zotero_articles or zotero_books is incorrect.
2023-11-10 08:18:06,885 - zotero_connect - ERROR - An error occurred during setup: Table structure is incorrect.
ERROR

ERROR: test_setup_with_no_tables (__main__.TestZoteroConnectPlugin)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/goma/miniconda3/envs/gpt2brain/lib/python3.10/unittest/mock.py", line 1379, in patched
    return func(*newargs, **newkeywargs)
  File "/tmp/ipykernel_29353/2085860152.py", line 35, in test_setup_with_no_tables
    self.plugin.setup(config={})
  File "/mnt/d/ai/cognimesh/backend/plugins/zotero_connect.py", line 99, in setup
    raise e
  File "/mnt/d/ai/cognimesh/backend/plugins/zotero_connect.py", line 92, in setup
    rai

<unittest.runner.TextTestResult run=1 errors=1 failures=0>

In [1]:
from backend.plugins.zotero_connect import ZoteroConnect
from backend.settings.backend_setting import store

# Create an instance of the plugin
zotero_plugin = ZoteroConnect()

# Assuming the 'store' object is your database connection from backend_setting
db_connection = store.connection

# Enable the plugin (if your system requires this step)
zotero_plugin.switch(enable=True)

# Set up the plugin, which includes table creation
zotero_plugin.setup(db_connection)

# You might want to catch any exceptions here and handle them appropriately
try:
    zotero_plugin.setup(db_connection)
except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during ZoteroConnect plugin setup: {e}")

try:
    # Disable the plugin first
    zotero_plugin.switch(enable=False)
    
    # Then call the delete method to clean up resources
    zotero_plugin.delete()

except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during ZoteroConnect plugin deletion: {e}")

2023-11-10 08:48:23,989 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 08:48:23,990 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 08:48:23,998 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 08:48:23,999 - zotero_connect - INFO - ZoteroConnect plugin setup completed successfully.
2023-11-10 08:48:23,990 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 08:48:23,998 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 08:48:23,999 - zotero_connect - INFO - ZoteroConnect plugin setup completed successfully.
2023-11-10 08:48:24,001 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 08:48:24,007 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 08:48:24,008 - zotero_connect - INFO - ZoteroConnect plugin setup completed successfully.
2023-11-10 08:48:24,010 - zotero_connect - INFO - ZoteroConnect plugin has been disabled.
20